### Inference 


In [ ]:
import torch
# from transformers import DNABertTokenizer, DNABertForSequenceClassification
from DNABERT.examples.transformers.data.processors.glue import DnaPromProcessor,glue_convert_examples_to_features
from DNABERT.examples.transformers.data.processors.utils import InputExample
from DNABERT.examples.run_finetune import TensorDataset
from transformers import BertConfig,    DNATokenizer,    BertForSequenceClassification
# 加载DNABERT模型和tokenizer
import pandas as pd
import numpy as np
import os
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 测试
# Read sequences from text file
sequences_pos = [];sequences_neg = []
with open('./dataset/your_test_set/positive.txt', 'r') as file:
    for line in file:
        sequence = line.strip()
        sequences_pos.append(sequence)
with open('./dataset/your_test_set/negative.txt', 'r') as file:
    for line in file:
        sequence = line.strip()
        sequences_neg.append(sequence)

df = pd.DataFrame({'name': [f'Sequence_{i}' for i, _ in enumerate(sequences_pos)],
                   'sequence': sequences_pos})
df['label']=1
# print(df)
df2 = pd.DataFrame({'name': [f'Sequence_{i}' for i, _ in enumerate(sequences_neg)],
                   'sequence': sequences_neg})
df2['label']=0
df=pd.concat([df,df2],axis=0)
def seq2kmer(seq):
    k=3
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers
def create_examples(df, set_type='dev'):
    examples = [];i=1
    for line in df.values:
        guid = "%s-%s" % (set_type, i)
        text_a = seq2kmer(line[0])
        label = str(line[1])
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        i=i+1
    return examples
def load_and_cache_examples(df):
    used_model="./results/models/bert/N_snrna/checkpoint-7200"# use your own model
    tokenizer = DNATokenizer.from_pretrained(used_model)
    processor = DnaPromProcessor()

    label_list = processor.get_labels()
    examples = (create_examples(df))   

    max_length = 101
    pad_on_left = 0
    pad_token = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    
    features = glue_convert_examples_to_features(
    examples,
    tokenizer,
    label_list=label_list,
    max_length=max_length,
    output_mode="classification",
    pad_on_left=pad_on_left, 
    pad_token=pad_token,
    pad_token_segment_id=0)
            
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    return all_input_ids, all_attention_mask, all_token_type_ids, all_labels
model=BertForSequenceClassification.from_pretrained("used_model").to(device)
token=load_and_cache_examples(df[["sequence","label"]])


class mydataset(Dataset):
    def __init__(self,df):
        self.df=df[["sequence","label"]]
        self.token=load_and_cache_examples(self.df)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        return tuple(tensor[idx] for tensor in self.token)
test_dataset=mydataset(df)

test_dataloader=DataLoader(test_dataset,batch_size=256,shuffle=False,drop_last=False)
model.eval()
test_result=[]


with torch.no_grad():
    for idx,i in enumerate(test_dataloader):
        x= {"input_ids": i[0].to(device), "attention_mask": i[1].to(device), "labels": i[3].to(device)}
        # print(x)
        outputs=model(**x)
        # print(outputs)
        test_result.append(outputs[1])
        # break
test_result=torch.cat(test_result,dim=0)
pred=test_result.softmax(dim=1).detach().cpu()
pred=torch.argmax(pred,dim=1)
y_true=df.label.values



In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
print(accuracy_score(y_true,pred))
print(precision_score(y_true,pred))
print(recall_score(y_true,pred))
print(f1_score(y_true,pred))
print(roc_auc_score(y_true,pred))